In [ ]:
import sqlite3

import numpy as np
import pandas as pd

Reading the raw dataset.

In [ ]:
connection = sqlite3.connect("../raw/commoncrawl_news_articles/articles.db")

df = pd.read_sql(
    "SELECT content_preprocessed, au.outlet_name FROM article_contents ac INNER JOIN article_urls au on ac.uuid = au.uuid WHERE language = 'en'",
    connection
)

outlets_df = pd.read_json(open("../raw/commoncrawl_news_articles/outlets.json"))
df = df.merge(outlets_df, how="outer", left_on="outlet_name", right_on="name")
df

Dropping useless columns.

In [ ]:
df = df.drop(columns=["outlet_name", "name", "tld", "filter", "allsides_name"])

Renaming columns.

In [ ]:
df = df.rename(columns={"content_preprocessed": "body", "allsides_rating": "leaning"})

Categorizing the leaning column.

In [ ]:
df["leaning"].unique()

In [ ]:
df["leaning"] = df["leaning"].replace({"lean left": "left", "lean right": "right"})
df["leaning"] = df["leaning"].astype("category")

Dropping rows with missing data.

In [ ]:
df = df.dropna()

Printing duplicates.

In [ ]:
df[df["body"].duplicated(keep=False)]

Dropping the duplicates.

In [ ]:
df = df.drop_duplicates(subset="body")

Inspecting body length.

In [ ]:
df["body_length"] = df["body"].str.len()
df["body_word_count"] = df["body"].str.split().str.len()
df = df.sort_values(by="body_length")
df.head()

In [ ]:
df.tail()

In [ ]:
# Ensuring to include the last (longest) item.
downsampled = pd.concat([df["body_length"].iloc[::100], df["body_length"].iloc[[-1]]]).drop_duplicates()
downsampled.plot.bar().xaxis.set_ticks([]);

Calculating the bounds for body length.

In [ ]:
q1 = np.percentile(df["body_length"], 25)
q3 = np.percentile(df["body_length"], 75)
iqr = q3 - q1
lower_bound_multiplier = 0.5
upper_bound_multiplier = 9
lower_bound = q1 - lower_bound_multiplier * iqr
upper_bound = q3 + upper_bound_multiplier * iqr
lower_bound, upper_bound

Removing the outliers based on the body length.

In [ ]:
df = df[(df["body_length"] >= lower_bound) & (df["body_length"] <= upper_bound)]
# Ensuring to include the last (longest) item.
downsampled = pd.concat([df["body_length"].iloc[::100], df["body_length"].iloc[[-1]]]).drop_duplicates()
downsampled.plot.bar().xaxis.set_ticks([]);

In [ ]:
df["body_length"].mean()

The leaning distribution.

In [ ]:
df.groupby("leaning", observed=True).size().plot.pie(autopct="%1.1f%%");

The distribution of body length sums per leaning.

In [ ]:
df.groupby("leaning", observed=True)["body_length"].sum().plot.pie(autopct="%1.1f%%", ylabel="");

Body length distribution of left leaning articles.

In [ ]:
df_leaning = df[df["leaning"] == "left"]
# Ensuring to include the last (longest) item.
downsampled = pd.concat([df_leaning["body_length"].iloc[::100], df_leaning["body_length"].iloc[[-1]]]).drop_duplicates()
downsampled.plot.bar().xaxis.set_ticks([]);

Body length distribution of center leaning articles.

In [ ]:
df_leaning = df[df["leaning"] == "center"]
# Ensuring to include the last (longest) item.
downsampled = pd.concat([df_leaning["body_length"].iloc[::100], df_leaning["body_length"].iloc[[-1]]]).drop_duplicates()
downsampled.plot.bar().xaxis.set_ticks([]);

Body length distribution of right leaning articles.

In [ ]:
df_leaning = df[df["leaning"] == "right"]
# Ensuring to include the last (longest) item.
downsampled = pd.concat([df_leaning["body_length"].iloc[::100], df_leaning["body_length"].iloc[[-1]]]).drop_duplicates()
downsampled.plot.bar().xaxis.set_ticks([]);

In [ ]:
df.to_parquet("../preprocessed/commoncrawl_news_articles.parquet")